# Assignment 3a Cognitive Modelling

Dit is onderdeel van de derde opdracht voor Cognitive Modelling, assignment 3 bestaat in totaal uit 3 onderdelen:

* __Assignment 3a (47 punten)__
* Assignment 3b (19 punten)
* Assignment 3c (23 punten)

__Let op__: Opdracht 3b en 3c werken alleen in Google Colab!

Geef antwoorden in blokken met code of met tekst.Gebruik voor antwoorden met tekst de ">" voor blockquotes en geef bij elke vraag ook __kort uitleg__ als hier om wordt gevraagd. __Let op__: soms staan er meerdere vragen bij een onderdeel, lees de tekst dus nauwkeurig. 

Sla het uiteindelijke notebook op met jullie studentnummers en achternamen in de filenaam: `studentnummer1_achternaam1_studentnummer2_achternaam2_opdrachtnummer`.

<br>

---

## Q-Learning:  State Spaces and Deep Q Networks

In de vorige opdrachten hebben we gekeken naar simpele taken waarbij de state space bekend was en de mogelijke acties ook. Vaak zijn omgevingen echter onbekend of een stuk ingewikkelder. We beginnen nu bij een kleine omgeving die verkend moet worden, maar die nog wel in een Q-table te vangen is. Let op dat steeds hierbij alle mogelijke handelingen wel duidelijk zijn, het is maar de vraag of dit voor biologische systemen ook het geval is.

We gaan ook kijken hoe we het klassieke Q-Learning (wat dus is gebaseerd op het Rescorla Wagner model) kunnen uitbreiden met functies die gebaseerd zijn op de werking van menselijke hersenen.

Uiteindelijk maken we een klein uitstapje naar Deep Q Networks: de huidige state-of-the-art methode in Reinforcement Learning, waar het basis Q-Learning idee wordt gecombineerd met dat van Deep Neural Nets, met name ConvNets. Aan het eind van deze opdracht moet je een goed idee hebben hoe deze Deep Q Nets werken, en ook begrijpen hoe je deze netwerken mogelijk kan aanpassen en ze slimmer te maken.

#### Winter is coming

Het is winter. Jij en je robotvriend waren in het park met een frisbee aan het spelen toen jij een wilde worp deed waardoor de frisbee midden op het meer terecht kwam. Het water is grotendeels goed bevroren, maar er zijn een paar zwakke plekken waar je door het ijs zakt en in het ijskoude water valt. <br>

![](Images\lake.png) <br>

Op dit moment is er een internationaal frisbeetekort, dus het is absoluut noodzakelijk dat je over het meer navigeert en de frisbee terughaalt. Omdat je zelf niet nat wilt worden, stuur je je robot om de frisbee te halen. Het oppervlak wordt beschreven met behulp van deze matrix:
    
        State:        State #:
        S F F F     0  1  2  3
        F H F H     4  5  6  7 
        F F F H     8  9  10 11 
        H F F G     12 13 14 15 
        
    S : start punt, safe
    F : bevroren surface, safe
    H : zwak ijs, val in het water (einde robot)
    G : Doel, de frisbee

Dit is de Frozenlake omgeving, een spel van de Open AI Gym. Dit spel wordt gebruikt als een soort benchmark om verschillende leeralgoritmes (vooral Q-learning) met elkaar te vergelijken. In deze Gym kan je nog veel meer leuke dingen vinden, en voor deze opdracht is het nodig om de gym library installeren.

`!pip install gymnasium` zou genoeg moeten zijn, kijk anders even op https://pypi.org/project/gymnasium/

Daarnaast gebruikt de gym ook nog wat elementen van `pygame` dus installeer dat ook:

`!pip install pygame`  of kijk op https://www.pygame.org/wiki/GettingStarted 

In [ ]:
# !pip install gymnasium

In [ ]:
# !pip install pygame

In [ ]:
# init gym and other libraries
import gymnasium as gym
import pygame
import math
import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

#### Loading the environment

In [ ]:
# We laden nu een 4x4 Frozen Lake. Slippery betekent dat de robot soms uitglijdt en een random kant op beweegt
# We start with non slippery ice 🤔
env_4x4 = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)

<br>

---

## 1. Q-Table learning

Het is jouw taak om de robot te leren veilig van start $S$ naar het doel $G$ te lopen door middel van Q learning. Daarmee voorkom je dat de robot in een wak valt.

Details over Q-learning zijn terug te vinden in de college slides en in het hoofdstuk van Gureckis & Love [**Computational reinforcement learning**](http://bradlove.org/papers/GureckisLovePress.pdf), voor meer verdieping in het online boek van [**Sutton & Barto**](http://incompleteideas.net/book/bookdraft2018jan1.pdf) en dan met name hoofdstuk 6.


**Let op:** In dit experiment zijn meerdere states. Dit heeft als gevolg dat bij het leren rekening gehouden moet worden met de actie die in een volgende state gemaakt wordt. We gaan er vanuit dat de robot altijd de actie kiest met de hoogste Q-value. De prediction error wordt dus:

$$\delta = r_{t+1} + \gamma\ max_a\ Q(s_{t+1} , a) − Q(s_t , a_t)$$

Waarbij de Q-value update nog steeds is gedefinieerd als:

$$Q(s_{t+1}, a_{t+1}) = Q(s_t, a_t) + \alpha \delta$$

In het begin van het experiment heeft de robot geen enkele kennis van de wereld en geen enkele verwachtingen voor het krijgen van beloningen. Voor elke actie ($L$: left, $R$: right, $U$: up, $D$: down) op elk vlak van de omgeving geldt dus:

$$Q(s, L) = Q(s, R) = Q(s, U) = Q(s, D) = 0$$

### Q1.a (8 punten)

Schrijf een functie `q_learn` die als input `env` (*Gymnasium* environment), `alpha` ($\alpha$), `gamma` ($\gamma$), `epsilon` ($\epsilon$), `epsilon_decay` en `num_episodes` accepteert.

De output van deze functie moet een table zijn met $Q$ waarden voor elke state en elke actie. Er zijn 16 states en 4 acties dus in totaal 64 combinaties. Voor de analyse van het algoritme is het ook belangrijk een lijst met totaal verdiende beloningen en de totale hoeveelheid stappen per episode als output te hebben. 

We gaan er nu vanuit dat de robot de $\epsilon$-greedy keuze regel toepast, waarbij $\epsilon$ afneemt met tijd. 

Hier alvast wat code om je op weg te helpen: 

```python
def q_learn(
    env: gym.Env,
    alpha: float = 0.8,
    gamma: float = 0.95,
    epsilon: float = 0.1,
    epsilon_decay: float = 0.999,
    num_episodes: int = 2000,
):
    # Initialize Q-table with all zeros
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    q_vals = np.zeros((n_states, n_actions))

    # Create arrays to contain total rewards and steps taken per episode
    total_rewards = np.zeros(num_episodes)
    num_steps = np.zeros(num_episodes, dtype=np.int32)

    # Start learning:
    for i in range(num_episodes):
        state, _ = env.reset()  # `state` is an integer, see the state numbers above
        episode_reward = 0  # Total rewards for this episode start at 0

        # The Q-Table learning algorithm, each episode consists of several learning trials.
        # If we reach the max of 100 trials (or steps), we start over.
        steps_elapsed = 0
        for steps_elapsed in range(1, 100):
            # First, implement the e-greedy to choose an action: if random.random() < epsilon: explore, else: exploit
            # As in previous assignments, draw randomly between maxima when there is more than one max Q

            # NOTE: actions in FrozenLake are defined as 0:left, 1:down, 2:right, 3:up
            a = ...  # TODO

            # Update the environment using env.step(0/1/2/3)
            # `done` is a boolean indicating whether next_state is a terminal state
            next_state, reward, done, _, _ = env.step(a)

            # Update Q-Table with new knowledge (assume you pick the action with the highest value in new state)
            q_vals[state, a] = ...  # TODO

            episode_reward += reward

            # TODO: If Frisbee or hole (`done`) then end trial

            state = next_state

        # Update `epsilon`, reducing exploration over episodes
        epsilon *= epsilon_decay

        num_steps[i] = steps_elapsed
        total_rewards[i] = episode_reward

    return total_rewards, num_steps, q_vals
```

De ingevoerde hyperparameters werken goed maar voel je vrij om hier mee te experimenteren. 


In [ ]:
# TO DO

<br>

### Q1.b   (5 punten)     

Laat de robot met behulp van deze functie 1000 episodes leren over deze wereld (1 leer episode bestaat dus uit max 100 trials). Bereken de gemiddelde score over alle episodes (heel soms komt dit model niet op de oplossing, run het dan nog eens).

In [ ]:
# TO DO

* Plot nu in bins van 10 episodes de verandering in gemiddelde beloning over tijd. Waar ligt het omslag punt?

Voor het plotten is het handig om onderstaande code te gebruiken.

```python
width = 10   # Width is de grote van elke bin
# Hier knippen we de laatste elementen (total_rewards.size % width) van de lijst
# Dan reshapen we naar een matrix van X * width en nemen we de mean over de width axis
# Hiermee krijgen we dus de gemiddelde beloning voor width episodes
result1 = total_rewards[:(total_rewards.size // width * width)].reshape(-1, width).mean(axis=1)
plt.plot(result1, label=...)
```

In [ ]:
# TO DO

> *Antwoord*

* Plot ook de gemiddelde hoeveelheid stappen die er gemaakt zijn in bins van 10. Hoe kort is de korste route tot succes waar het model op uitkomt? Hoe verhoudt dit zich tot de werkelijke korste route?

In [ ]:
# TO DO

> *Antwoord*

<br> 

### Q1.c (2 punten)

Laten we nu naar de $Q$ tables gaan kijken. Zo kunnen we inspecteren welke route de robot zou lopen als deze altijd de max $Q$ zou kiezen (dus puur greedy, $\epsilon$ = 0).

Gebruik de final $Q$ table die `q_learn()` aan het eind geeft. Hierbij is elke rij een state, met de vier acties die mogelijk zijn. Dus `output[2][9,:]` geeft de Q values van de vier acties die je kan doen in state 9. 

Zorg dat je een lijst krijgt ('Qmax') met voor elke state de actie die de hoogste Q value heeft. Representeer hier elke actie met een getal (0 = links, 1 = beneden, 2 = rechts, 3 = boven). Plot vervolgens deze lijst in een grid: 

```python
print("0=links, 1=beneden, 2=rechts, 3=boven")
Qmax=np.asarray(Qmax).reshape((4,4))
print(Qmax)

```

* Ga nu na hoe de robot loopt, gaat deze links of rechts langs het wak linksboven op de kaart?

In [ ]:
# TO DO

> *Antwoord*

<br>

---

## 2. Changes in the environment

We kijken nu naar hoe de robot zich op glad ijs beweegt. In deze omgeving zal hij dus af en toe een willekeurige kant op schuiven. Laad de omgeving opnieuw met `is_slippery = True`. Deze omgeving is als trainingsomgeving bedacht omdat er vaak ruis (noise) zit in de omgeving (of in perceptie zelf) en soms dingen dus net anders lopen dan verwacht. Een robust algortime kan hier dan ook mee leren omgaan. 

Het gebruiken van een algoritme dat in een simulatie van een perfecte wereld is getest, zoals hierboven, kan tot catastrofale uitkomsten leiden in de echte wereld (waarbij de weg ineens glad kan zijn of er een sterke storm opsteekt, er is altijd wel iets onvoorzien).

In [ ]:
# let's load the Frozen lake environment
# slippery means that sometimes the robot will slip on the ice and move in a random direction
env_slip = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True)

<br>

### Q2.a (5 punten)
Run hetzelfde `q_learn` algoritme met dezelfde parameterwaardes als hierboven. 
* Laat de gemiddelde score zien, en hoe de gemiddelde beloning en het aantal stappen met de tijd verandert. 
* Vergelijk dit met de non-slippery omgeving van hiervoor, noem de meest opvallende verschillen en leg uit waarom we die zien. 

In [ ]:
# TO DO

> *Antwoord*

### Loading the 8x8 environment

We gaat het nu wat moeilijker maken en maken het bevroren meer wat groter. Laad nu het **niet gladde** grotere ijsmeer in:
```python
env = gym.make('FrozenLake-v1',map_name="8x8",is_slippery=False)
```

In [ ]:
# let's load the Frozen lake environment
env_8x8 = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=False)

### Q2.b (4 punten)
Het nieuwe meer is 8x8 maar verder helemaal hezelfde. We kunnen dan ook weer hetzelfde `q_learn()` hier op los laten. Doe dit, met weer dezelfde parameterwaardes als je eerder bij de 4x4 hebt gebruikt. 
* Rapporteer wederom hoe goed de robot het doet in termen van verkregen beloningen en aantal stappen genomen. 
* Zie je ook een omslag punt? (probeer een paar keer als het model niet lijkt te leren)

In [ ]:
# TO DO

> *Antwoord*

---

## 3. Introducing Intuitive Physics

In het grid van 8x8 zien we dat de robot veel trials nodig heeft om een simpele taak op te lossen. En dit is nog maar een erg simpele wereld met slechts 64 states! We gaan nu proberen of we de Q-learning robot een beetje kunnen helpen sneller te leren. Zoals we in het college gezien hebben (en zie [hier](https://arxiv.org/pdf/1802.10217.pdf)) zijn mensen heel snel in het leren van computer games omdat ze hun kennis over de wereld toepassen op de spelwereld. Laten we dat hier ook proberen.

### Q3.a (10 punten)

Implementeer een nieuwe `q_learn_intuition` met drie toevoegingen die je makkelijk aan of uit kunt zetten, zodat je het algoritme met en zonder deze toevoeging kunt testen:
* Negatieve reward
* Muren
* Cognitieve map

__Negatieve reward__ <br>
Om te beginnen is het voor de robot niet heel duidelijk dat in een wak vallen een slecht idee is. De beloning die er bij hoort is nu nul. Het is voor mensen duidelijk dat in een wak vallen niet goed is voor het behalen van het doel. Zo kan je dus de volgende kleine aanpassing maken:

```python
 if done and reward == 0:
     reward = -1
```

Dit moet direct na de `env.step()` stap waarbij `reward` is bepaald. Door een negatieve reward te gebruiken zal het model sneller leren wakken te vermijden.

__Muren__ <br>
Het is nu voor de robot nog mogelijk om tegen muren aan te botsen zonder daar iets over te leren. Wanneer de robot bijvoorbeeld vanaf state 3 (in de rechter boven hoek) naar rechts of boven probeert te lopen, zal de nieuwe state weer 3 zijn. Leer de robot af om dit te doen.

__Cognitieve map__ <br>
Leer de robot daarnaast dat als je in een wak loopt, je ook in dit wak loopt als je van een andere kant naar hetzelfde hokje loopt. Stel je loopt van boven in een wak, leer dan meteen dat je ook in datzelfde wak loopt als je er van links, rechts of van onder in loopt. Met andere woorden: introduceer een minimale intuitive physics! <br>
   
   *Hint: Houd hierbij rekening met dat sommige wakken tegen de randen aanzitten, update geen states die niet bestaan*

Je kan naar de Q tables kijken om te zien of jouw implementatie gewerkt heeft.

In [ ]:
def q_learn_intuition():
    # TO DO
    return total_rewards, num_steps, q_vals

### Q3.b (4 punten)

Run de verschillende implementaties van het model met geen, een of een combinatie van elementen van kennis. Run elk model 20 keer voor 500 episodes en vergelijk de average rewards. Run alles voor de zekerheid een paar keer om te voorkomen dat er door toevalligheid een raar resultaat uit komt.  Beschrijf hoe elk element wel of niet bijdraagt aan beter of sneller leren. 

De output moet er dus ongeveer zo uit zien:

```python
Baseline average score: 
Neg_reward average score: 
Muren average score: 
Cog_map average score: 
Neg_reward + muren average score: 
Neg_reward + cog_map average score:
Muren + cog_map average score:
Neg_reward + muren + cog_map  average score:
```

In [ ]:
# TO DO

> *Antwoord*

### Q3.c  (3 punten)

Laat van het beste model (run een aantal keer om te besluiten welke dat is) ook nog de grafieken van beloning en pad lengte zien, zodat je die kan vergelijken met die hierboven voor het baseline model. Beschrijf wat je ziet.

In [ ]:
# TO DO

> *Antwoord*

### Q3.d More common sense (6 punten)
Verzin zelf nog een ingreep die het model beter en sneller zou kunnen laten leren. Denk dan vooral na over hoe mensen, zoals jij, het spel zouden spelen. Laat zien of, en zo ja hoeveel beter jouw model werkt. 

Laat zien hoe jouw implementatie een verbetering is op het baseline model, maar ook of het nog wat toevoegd aan het best model van vraag Q3.b. 

In [ ]:
# TO DO

> *Antwoord*

## Q Networks

We hebben nu gewerkt met een kleine wereld met maar maximaal 64 states. De echte wereld, en veel leuke spelletjes, hebben natuurlijk een veel grotere state space, en dan wordt het al snel erg lastig om nog een Q table te gaan bijhouden. Dit is waar Q netwerken heel handig zijn, en dan met name Deep Q Networks (DQN). Een deep neural network kan helpen de state space een stuk beter generaliseerbaar te maken, en vergelijkbare Q-values toekennen aan states die veel op elkaar lijken, wat een hele waardevolle eigenschap blijkt (zie bijv. de oorspronkelijke DQN paper over Atari games leren spelen [hier](https://arxiv.org/pdf/1312.5602v1.pdf)).

We gaan een Deep Q Netwerk in elkaar zetten in Google Colab. Open daarvoor eerst `Assignment_3_Uitleg_Colab`.